## Reddit Scraping
Documentation reference: https://praw.readthedocs.io/en/stable/

In [40]:
import requests
import pandas as pd
import praw
from praw.models import MoreComments

import os
import zipfile

os.makedirs("../output", exist_ok=True)
from dotenv import load_dotenv
load_dotenv()


True

# Scrape post from related subreddits
Usage: Consumer sentiment analysis 

In [43]:

reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
)

# "Skincare_Addiction", "asianskincare", "Blackskincare","SkincareAddicts"
subreddit_list = ["SkincareAddiction",]

all_posts = []

# get the top 20 post from each subreddit (don't know rate limit so 20 for now)
# Documentation: https://praw.readthedocs.io/en/stable/code_overview/models/subreddit.html
def get_top_comments(post):

    post.comments.replace_more(limit=0)  

    top_comments = []
    for comment in post.comments:
        if isinstance(comment, MoreComments):
            continue 
        top_comments.append(comment.body)
        if len(top_comments) == 5:
            break
    return top_comments
        
for sub in subreddit_list:
    try:
        post_collection = reddit.subreddit(sub).top(limit=20)  
        for post in post_collection: 
            top_comments = get_top_comments(post)
            all_posts.append({
            "subreddit_name": sub,
            "post_id": post.id,
            "title": post.title,
            "description": post.selftext,
            "score": post.score,
            "num_comments": post.num_comments,
            "top_comments": top_comments,
            "upvote_ratio": post.upvote_ratio,
           })
    except Exception as e:
        print(f"Error scraping {sub}: {e}")
        continue
    
subreddit_df = pd.DataFrame(all_posts)
subreddit_df
subreddit_df.to_csv("../output/subreddit_data.csv", index=False)



In [46]:
skincare_df = pd.read_csv("../output/subreddit_data.csv")
skincare_df.head()


"['IM JUST SO FUCKING HAPPY FOR YOU. OH MY GOD IM SO EXCITED YOU LOOK BEAUTIFUL 😍', 'For my grad makeup I’m wearing a light coverage foundation just to cover the scars, but the difference in texture of my skin in both pics is so so astounding!! I started accutane January 2nd, 2018, and just finished it on June 4th. Still got those dry lips though. \\n\\nFor my routine I’ve kept it pretty simple:\\n \\nAM\\nCetaphil Gentle Skin Cleanser\\nCetaphil Daily Facial Mosturizer with SPF\\n\\nPM\\nCetaphil Gentle Skin Cleanser\\nThayers Lavender Witch Hazel\\nCetaphil Daily Facial Mosturizer with SPF\\n\\nEdit : Wow I just want to thank everyone for their really kind words. I really appreciate all the support and the positivity being spread around, it’s such an amazing feeling to be happy with your own skin again!! \\n\\nI’ve been having problems with acne for about 5 years now, and for anyone asking or worrying about accutane, I personally wish I’d started it sooner. Of course, I’m not an expe